Books Recommendation system - Collaborative filtering method

This notebook is item(books) based recommendation system, which uses books ratings from the dataset to suggest new books to users.


In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [2]:
books = pd.read_csv('books.csv')

In [3]:
books


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...


In [4]:
books['original_title'].isnull().sum() #check if any title has null value

585

In [5]:
books['book_id'].isnull().sum() #check if any book_id has null value

0

In [6]:
ratings = pd.read_csv('ratings.csv') #import ratings into dataframe

In [7]:
ratings.head()


,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


check for missing values and none values

In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981756 entries, 0 to 981755
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   book_id  981756 non-null  int64
 1   user_id  981756 non-null  int64
 2   rating   981756 non-null  int64
dtypes: int64(3)
memory usage: 22.5 MB


In [9]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [10]:
books['title']

0                 The Hunger Games (The Hunger Games, #1)
1       Harry Potter and the Sorcerer's Stone (Harry P...
2                                 Twilight (Twilight, #1)
3                                   To Kill a Mockingbird
4                                        The Great Gatsby
                              ...                        
9995                            Bayou Moon (The Edge, #2)
9996    Means of Ascent (The Years of Lyndon Johnson, #2)
9997                                The Mauritius Command
9998    Cinderella Ate My Daughter: Dispatches from th...
9999                                  The First World War
Name: title, Length: 10000, dtype: object

In [11]:
books_dataset = pd.DataFrame(books, columns=['book_id', 'authors', 'title', 'average_rating'])

In [12]:
books_dataset.head()

,book_id,authors,title,average_rating
0,2767052,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",4.34
1,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,4.44
2,41865,Stephenie Meyer,"Twilight (Twilight, #1)",3.57
3,2657,Harper Lee,To Kill a Mockingbird,4.25
4,4671,F. Scott Fitzgerald,The Great Gatsby,3.89


In [15]:
books_dataset = books_dataset.sort_values('book_id') #sort books id's

In [16]:
books_dataset['book_id'] #id;s are sorted

26             1
20             2
1              3
17             5
23             6
          ...   
7522    31538647
4593    31845516
9568    32075671
9579    32848471
8891    33288638
Name: book_id, Length: 10000, dtype: int64

merging both the books dataset and ratings dataset on - book_id files

In [17]:
books_data = pd.merge(books_dataset, ratings, on='book_id')

In [18]:
books_data

,book_id,authors,title,average_rating,user_id,rating
0,1,"J.K. Rowling, Mary GrandPré",Harry Potter and the Half-Blood Prince (Harry ...,4.54,314,5
1,1,"J.K. Rowling, Mary GrandPré",Harry Potter and the Half-Blood Prince (Harry ...,4.54,439,3
2,1,"J.K. Rowling, Mary GrandPré",Harry Potter and the Half-Blood Prince (Harry ...,4.54,588,5
3,1,"J.K. Rowling, Mary GrandPré",Harry Potter and the Half-Blood Prince (Harry ...,4.54,1169,4
4,1,"J.K. Rowling, Mary GrandPré",Harry Potter and the Half-Blood Prince (Harry ...,4.54,1185,4
...,...,...,...,...,...,...
79696,9998,"Kōbō Abe, E. Dale Saunders",The Woman in the Dunes,3.91,51295,5
79697,9998,"Kōbō Abe, E. Dale Saunders",The Woman in the Dunes,3.91,51559,5
79698,9998,"Kōbō Abe, E. Dale Saunders",The Woman in the Dunes,3.91,52087,4
79699,9998,"Kōbō Abe, E. Dale Saunders",The Woman in the Dunes,3.91,52330,4


create a dataframe with rating of every user for every book

In [19]:
book_rating = pd.pivot_table(books_data, index='user_id', values='rating', columns='title', fill_value=0)

In [20]:
book_rating

title,'Salem's Lot,"'Tis (Frank McCourt, #2)",1421: The Year China Discovered America,1776,1984,A Bend in the River,A Bend in the Road,A Brief History of Time,A Briefer History of Time,A Case of Need,...,"Women in Love (Brangwen Family, #2)",World War Z: An Oral History of the Zombie War,"World Without End (The Kingsbridge Series, #2)",Wuthering Heights,"Xenocide (Ender's Saga, #3)",Year of Wonders,You Shall Know Our Velocity!,Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values,Zodiac,number9dream
user_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
53420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0
53422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0


get correlation score of each book
transpose is used to get correlation of every book
not users

let's find books recommendation for a book - "'Salem's Lot"

In [21]:
rat=pd.DataFrame(books_data.groupby('title')['rating'].mean()) #

Create a dataframe for each book's average rating and total ratings

In [22]:
rat['num of ratings'] = pd.DataFrame(books_data.groupby('title')['rating'].count())#
rat.head()

,rating,num of ratings
title,,
'Salem's Lot,3.932432,74
"'Tis (Frank McCourt, #2)",3.970000,100
1421: The Year China Discovered America,3.750000,100
1776,3.930000,100
1984,3.170000,100


In [23]:
salems_lot_userrating=book_rating["'Salem's Lot"]

In [24]:
salems_lot_userrating.head()

user_id
2    0
3    0
4    0
7    0
9    0
Name: 'Salem's Lot, dtype: int64

In [25]:
similar_to_salem=book_rating.corrwith(salems_lot_userrating) 
similar_to_salem   #This is a correlation of 'salems_lot' book ratings with other books ratings given by users

title
'Salem's Lot                                                         1.000000
'Tis (Frank McCourt, #2)                                            -0.002777
1421: The Year China Discovered America                             -0.002759
1776                                                                -0.002790
1984                                                                -0.002741
                                                                       ...   
Year of Wonders                                                     -0.002764
You Shall Know Our Velocity!                                        -0.002751
Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values   -0.002741
Zodiac                                                              -0.002759
number9dream                                                        -0.002747
Length: 812, dtype: float64

In [26]:
corr_salem= pd.DataFrame(similar_to_salem,columns=['Correlation'])#now, create dataframe for above correlation
corr_salem.dropna(inplace=True)

In [27]:
corr_salem.head()

,Correlation
title,
'Salem's Lot,1.000000
"'Tis (Frank McCourt, #2)",-0.002777
1421: The Year China Discovered America,-0.002759
1776,-0.002790
1984,-0.002741


In [28]:
corr_salem = corr_salem.join(rat['num of ratings'])# merge this correlation with total ratings for each book by users
corr_salem.head()

,Correlation,num of ratings
title,,
'Salem's Lot,1.000000,74
"'Tis (Frank McCourt, #2)",-0.002777,100
1421: The Year China Discovered America,-0.002759,100
1776,-0.002790,100
1984,-0.002741,100


Following are books recommended for 'Salem's lot' book.

In [29]:
corr_salem[(corr_salem['Correlation']>=0.10)&(corr_salem['num of ratings']>=50)] # books having corrlation >0.10 are recommended for reading.

,Correlation,num of ratings
title,,
'Salem's Lot,1.000000,74
The Man From St. Petersburg,0.124407,68


Now following code shows recommendation for book 'The Alchemist'

In [32]:
Alchemist_user_ratings = book_rating['The Alchemist']
similar_to_Alchemist =  book_rating.corrwith(Alchemist_user_ratings)
corr_Alchemist = pd.DataFrame(similar_to_Alchemist,columns=['Correlation'])
corr_Alchemist.dropna(inplace=True)
corr_Alchemist = corr_Alchemist.join(rat['num of ratings'])

In [33]:
corr_Alchemist

,Correlation,num of ratings
title,,
'Salem's Lot,-0.002772,74
"'Tis (Frank McCourt, #2)",0.012097,100
1421: The Year China Discovered America,-0.003253,100
1776,0.024311,100
1984,0.012405,100
...,...,...
Year of Wonders,0.039942,100
You Shall Know Our Velocity!,-0.003244,100
Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values,0.003050,100


In [34]:
corr_Alchemist[(corr_Alchemist['Correlation']>=0.10)&(corr_Alchemist['num of ratings']>=50)]

,Correlation,num of ratings
title,,
Getting Things Done: The Art of Stress-Free Productivity,0.113833,100
Notes from a Small Island,0.104721,100
Perfume: The Story of a Murderer,0.137695,100
"Sex, Drugs, and Cocoa Puffs: A Low Culture Manifesto",0.146357,100
The Alchemist,1.000000,100
The New York Trilogy,0.219666,100
The Plot Against America,0.580486,100
The Virtue of Selfishness: A New Concept of Egoism,0.138701,100
The Zahir,0.152514,100


In [ ]:
#General Function for perticular book recommendation

In [36]:
def get_recommendation(books):
    book_user_ratings = book_rating[books]
    similar_to_book =  book_rating.corrwith(book_user_ratings)
    corr_book = pd.DataFrame(similar_to_book,columns=['Correlation'])
    corr_book.dropna(inplace=True)
    corr_book = corr_book.join(rat['num of ratings'])
    #print(corr_book)
    corr_book[(corr_book['Correlation']>=0.10)&(corr_book['num of ratings']>=50)]
    print(corr_book[(corr_book['Correlation']>=0.10)&(corr_book['num of ratings']>=50)].index)

In [40]:
x=str(input("Enter book name - "))

Enter book name - The Alchemist


In [41]:
y=get_recommendation(x)


Index(['Getting Things Done: The Art of Stress-Free Productivity',
       'Notes from a Small Island', 'Perfume: The Story of a Murderer',
       'Sex, Drugs, and Cocoa Puffs: A Low Culture Manifesto', 'The Alchemist',
       'The New York Trilogy', 'The Plot Against America',
       'The Virtue of Selfishness: A New Concept of Egoism', 'The Zahir',
       'Treasure Island'],
      dtype='object', name='title')


In [ ]:
Above are books recommended for entered book name - x 
[Here I have shown for The Alchemist. Anyone can change input x and get recommendation]